In [1]:
# make hydra scGOclust object using eggnog-mapper GO annotations and scRNA-seq data
# © EMBL-European Bioinformatics Institute, 2025
# Yuyao Song <ysong@ebi.ac.uk>
# Mar 2025


In [35]:
library(tidyverse)
library(Seurat)
library(scGOclust)

In [36]:
obj <- readRDS("./Hydra_vulgaris_whole_genome_seurat_obj.rds")

In [37]:
obj

An object of class Seurat 
17089 features across 24458 samples within 1 assay 
Active assay: RNA (17089 features, 0 variable features)
 2 layers present: counts, data

In [38]:
head(obj@assays$RNA@meta.features)

,gene_name_full,gene_id_short
,<chr>,<chr>
g10012.t1,g10012.t1|ZSWM1_HUMAN,g10012.t1
g10014.t1,g10014.t1|ELL_HUMAN,g10014.t1
g10017.t1,g10017.t1|TR112_BOVIN,g10017.t1
g10018.t1,g10018.t1|COFI_GIBZE,g10018.t1
g1002.t1,g1002.t1|LRC71_MOUSE,g1002.t1
g10020.t1,g10020.t1|KCNQ1_XENLA,g10020.t1


In [39]:
gene_name_to_go = read_tsv("./hydra2.0_genome_protein_non_electronic_GO.tsv", col_names = FALSE)

Rows: 22067 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): X1, X2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [40]:
head(gene_name_to_go)

X1               
1 Sc4wPfr_97.g10.t1
2 Sc4wPfr_97.g13.t1
3 Sc4wPfr_97.g14.t1
4 Sc4wPfr_97.g15.t1
5 Sc4wPfr_97.g17.t1
6 Sc4wPfr_97.g18.t1
  X2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [41]:
gene_name_to_go = gene_name_to_go %>% separate_rows(X2, sep = ',') %>% filter(X2 != '-') %>% 
rename(external_gene_name = X1, go_id = X2)

In [73]:
bp_terms = read_tsv("../go_ontology/biological_process_go_terms.txt", col_names = FALSE)
gene_name_to_go_bp = gene_name_to_go %>% filter(go_id %in% bp_terms$X1)  %>% mutate(go_category = 'biological_process')

Rows: 28050 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): X1, X2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


external_gene_name,go_id,go_category
<chr>,<chr>,<chr>
Sc4wPfr_97.g13.t1,GO:0008150,biological_process
Sc4wPfr_97.g13.t1,GO:0009892,biological_process
Sc4wPfr_97.g13.t1,GO:0009894,biological_process
Sc4wPfr_97.g13.t1,GO:0010468,biological_process
Sc4wPfr_97.g13.t1,GO:0010605,biological_process
Sc4wPfr_97.g13.t1,GO:0010608,biological_process


[1] 896409      3

In [74]:
gene_name_to_go_bp = merge(gene_name_to_go_bp, bp_terms, by.x = 'go_id', by.y = 'X1')

[1] 896409      4

[1] 896409      4

In [75]:
gene_name_to_go_bp = gene_name_to_go_bp %>% rename(name_1006 = X2)

In [78]:
gene_name_to_go_bp = gene_name_to_go_bp %>% 
mutate(gene_id =  gsub(".*?(g\\d+.*)", "\\1", external_gene_name))

In [82]:
gene_name_to_go_bp %>% write_tsv("Hvul_whole_genome_eggnog_name_to_go_id.tsv")

In [83]:
seurat_obj <- obj
ensembl_to_GO <- gene_name_to_go_bp
feature_type = 'gene_id'

In [84]:
  message("collect data")
  counts <- as.matrix(seurat_obj@assays$RNA@counts)

collect data

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 3.1 GiB”


In [85]:
head(rownames(counts))

[1] "g10012.t1" "g10014.t1" "g10017.t1" "g10018.t1" "g1002.t1"  "g10020.t1"

In [86]:
  if(!(feature_type %in% colnames(ensembl_to_GO))){
    stop(paste0(feature_type, " is not in colnames(ensembl_to_GO), please check the var type"))
  }

In [87]:
  ## pivot GO to feature type table to matrix
  go_matrix <- ensembl_to_GO %>%
    dplyr::mutate(placehold = 1) %>%
    tidyr::pivot_wider(id_cols = eval(feature_type), names_from = name_1006, values_from = placehold, values_fill = 0, values_fn = dplyr::first)

In [88]:
# this way of pivoing, if the gene WB id has a GO annot in any peptide, it will be 1

In [89]:
head(go_matrix)

gene_id,mitochondrion inheritance,mitochondrial genome maintenance,reproduction,vacuole inheritance,single strand break repair,regulation of DNA recombination,regulation of mitotic recombination,mitotic spindle elongation,maltose metabolic process,⋯,regulation of unsaturated fatty acid biosynthetic process,positive regulation of unsaturated fatty acid biosynthetic process,regulation of BMP secretion,regulation of caveolin-mediated endocytosis,positive regulation of caveolin-mediated endocytosis,malonyl-CoA metabolic process,malonyl-CoA catabolic process,lipoxin metabolic process,lipoxin biosynthetic process,lysobisphosphatidic acid metabolic process
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
g6399.t1,1,0,1,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
g14402.t1,0,1,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
g27067.t1,0,1,0,0,0,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
g22330.t1,0,1,1,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
g21689.t1,0,1,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
g22024.t1,0,1,1,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [91]:
  shared <- intersect(go_matrix[[feature_type]], rownames(counts))

In [96]:
go_matrix <- go_matrix %>%
    dplyr::filter(get(feature_type) %in% shared) %>%
    dplyr::arrange(get(feature_type))
  go_matrix <- go_matrix %>% tibble::column_to_rownames(eval(feature_type))

In [98]:
  counts <- t(counts)
  counts <- counts %>% as.data.frame()
  counts <- counts %>% dplyr::select(eval(shared))
  counts <- counts[, match(rownames(go_matrix), colnames(counts))]

In [100]:
  if (!(all(colnames(counts) == rownames(go_matrix)))) {
    stop("error during calculation, please check input format")
  }

In [101]:
  message("compute GO to cell matrix, might take a few secs")
  start_time <- Sys.time()
  go_mtx <- Matrix::Matrix(as.matrix(counts), sparse = TRUE) %*% Matrix::Matrix(as.matrix(go_matrix), sparse = TRUE)
  go_obj <- Seurat::CreateSeuratObject(counts = t(as.matrix(go_mtx)), meta.data = seurat_obj@meta.data)
  end_time <- Sys.time()

  message(paste0("time used: ", round(end_time - start_time, 2), " secs"))

compute GO to cell matrix, might take a few secs

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.4 GiB”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
time used: 1.06 secs



In [102]:
go_obj

An object of class Seurat 
13038 features across 24458 samples within 1 assay 
Active assay: RNA (13038 features, 0 variable features)
 2 layers present: counts, data

In [103]:
  all_zero_terms = names(rowSums(as.matrix(go_obj@assays$RNA@counts))[which(rowSums(as.matrix(go_obj@assays$RNA@counts)) == 0)])
  message(paste0("removing ", length(all_zero_terms)), " all zero terms")

  go_obj <- go_obj[which(!(rownames(go_obj) %in% all_zero_terms)), ]

  message("returning GO Seurat object")

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.4 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.4 GiB”
removing 83 all zero terms

returning GO Seurat object



In [104]:
go_obj

An object of class Seurat 
12955 features across 24458 samples within 1 assay 
Active assay: RNA (12955 features, 0 variable features)
 2 layers present: counts, data

In [105]:
saveRDS(go_obj, "Hvul_go_object_whole_genome.rds")